In [1]:
#----Example of a summary---
#---First iteration, not a great model-----

In [2]:
import spacy
# Need to run !python -m spacy download en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [3]:
from pathlib import Path as pathl
import sys
import re
from heapq import nlargest

In [4]:
#append relevant file paths
new_path = pathl('.')
parent = new_path.resolve().parent
sys.path.append(str(parent))
from pdf_parser import pipeline

In [5]:
#give the filepath to the preprocessing pipeline
data_path = parent / 'Data'
pdf_file = data_path /'coke_1.pdf'
text_dict = pipeline(filepath = str(pdf_file))

In [6]:
#convert to a single string for ease of completion
#---Incorporate the titles later---
text = list(text_dict.values())
text = sum(text, [])
text = ' '.join(text)

In [7]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
tokens = [token.text for token in doc]
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~\n'

In [20]:
#Remove stopwords, punctuation and add the remaining to a dictionary with num occurances as values
stopwords = list(STOP_WORDS)
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

In [21]:
#TF-IDF
max_frequency = max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

In [23]:
#Sentence scores gives the sum of the normalized frequencies of the words in a given sentence
sentence_tokens = [sent for sent in doc.sents]
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequencies[word.text.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.text.lower()]

In [33]:
#Selects only a few of the most relevant sentences. In this case the top 5% of sentences
select_length = int(len(sentence_tokens)*0.05)
summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)

In [34]:
final_summary = [word.text for word in summary]
summary = ' '.join(final_summary)

In [35]:
summary

'We prohibit the  use of all forms of forced labour, including prison  labour, indentured labour, bonded labour, military  labour, slave labour and any form of human trafficking  within our company and by any company that directly  supplies or provides services to our business.         Developed in partnership with The Coca-Cola Company, our SAGPs cover the Coca-Cola  systemÕs key agricultural ingredients, and define the standards we expect our agricultural  ingredient suppliers to adhere to in terms of human and workplace rights Ð including  prohibitions on modern slavery and child labour, the environment, and management  systems.   Developed in partnership with The Coca-Cola Company, our  SAGPs cover the Coca-Cola systemÕs key agricultural  ingredients, and define the standards we expect our  agricultural commodity suppliers to adhere to in terms of  human and workplace rights, the environment, and  management systems.'